# Statistic inference 

In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from matplotlib.patches import FancyArrowPatch
def find_project_root(start_path=None):
    """查找项目根目录（包含data和function目录的目录）"""
    if start_path is None:
        start_path = Path.cwd()
    
    current = Path(start_path).resolve()
    
    # 向上查找，直到找到包含data和function目录的目录
    for _ in range(5):  # 最多向上查找5层
        if (current / 'data').exists() and (current / 'function').exists():
            return current
        parent = current.parent
        if parent == current:  # 到达根目录
            break
        current = parent
    
    # 如果找不到，假设当前目录的父目录是项目根目录
    return Path.cwd().parent

project_root = find_project_root()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

DATA_PATH = project_root / 'data'

print(f"项目根目录: {project_root}")
print(f"数据路径: {DATA_PATH}")



项目根目录: C:\Dev\Landuse_Zhong_clean
数据路径: C:\Dev\Landuse_Zhong_clean\data


In [2]:
import pandas as pd 

csv_path = DATA_PATH / 'US_data/US_analysis_reslut/state_integration_analysis_simplified.csv'
state_analysis_path = DATA_PATH / 'US_data/US_analysis_reslut/state_level_analysis.csv'

state_intergrations = pd.read_csv(csv_path)
state_analysis = pd.read_csv(state_analysis_path)

In [5]:
state_intergrations

,State_name,Solution_Type,Environmental_sustainability,Emission_mitigation_ability,Economic_feasibility
0,Alabama,Economic,51866.832235,2.863123e+08,-3.844611e+09
1,Alabama,Emission_mitigation,60042.752401,3.311914e+08,-4.541858e+09
2,Alabama,Environmental,61823.883510,3.243282e+08,-4.841558e+09
3,Alabama,WCCD,52292.254042,2.749749e+08,-3.922319e+09
4,Arizona,Economic,26622.538454,1.880112e+08,3.262850e+10
...,...,...,...,...,...
187,Wisconsin,WCCD,47604.826564,2.087652e+08,-3.664799e+10
188,Wyoming,Economic,15573.252326,1.490355e+08,1.338115e+10
189,Wyoming,Emission_mitigation,11710.541729,1.127247e+08,1.081795e+10
190,Wyoming,Environmental,5467.586405,3.642128e+07,2.908299e+09
